In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
import pandas as pd

data = pd.read_csv("/kaggle/input/cattells-16-personality-factors/16PF/data.csv",delimiter="\t")

In [9]:
data

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,P7,P8,P9,P10,age,gender,accuracy,country,source,elapsed
0,1,4,2,3,3,2,3,4,4,3,...,1,2,5,2,17,1,92,US,6,914
1,4,3,4,3,4,4,4,4,2,2,...,3,2,2,2,37,1,100,US,1,891
2,3,4,4,4,4,4,4,3,2,2,...,2,3,3,3,31,1,80,US,6,903
3,4,5,4,4,4,3,3,2,2,2,...,2,3,4,4,32,1,93,US,1,806
4,4,0,4,4,4,3,5,1,2,4,...,4,2,3,2,46,2,87,NZ,1,1826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,24,1,100,IN,1,274
49155,4,5,4,4,4,3,5,3,2,3,...,3,4,4,4,22,1,50,ZA,1,584
49156,2,4,2,3,3,3,4,4,2,4,...,3,4,5,4,29,2,85,BE,6,932
49157,4,4,4,4,4,4,4,4,1,3,...,2,4,5,4,48,1,95,CZ,2,1212


In [6]:
data

,"A1\t""A2""\t""A3""\t""A4""\t""A5""\t""A6""\t""A7""\t""A8""\t""A9""\t""A10""\t""B1""\t""B2""\t""B3""\t""B4""\t""B5""\t""B6""\t""B7""\t""B8""\t""B9""\t""B10""\t""B11""\t""B12""\t""B13""\t""C1""\t""C2""\t""C3""\t""C4""\t""C5""\t""C6""\t""C7""\t""C8""\t""C9""\t""C10""\t""D1""\t""D2""\t""D3""\t""D4""\t""D5""\t""D6""\t""D7""\t""D8""\t""D9""\t""D10""\t""E1""\t""E2""\t""E3""\t""E4""\t""E5""\t""E6""\t""E7""\t""E8""\t""E9""\t""E10""\t""F1""\t""F2""\t""F3""\t""F4""\t""F5""\t""F6""\t""F7""\t""F8""\t""F9""\t""F10""\t""G1""\t""G2""\t""G3""\t""G4""\t""G5""\t""G6""\t""G7""\t""G8""\t""G9""\t""G10""\t""H1""\t""H2""\t""H3""\t""H4""\t""H5""\t""H6""\t""H7""\t""H8""\t""H9""\t""H10""\t""I1""\t""I2""\t""I3""\t""I4""\t""I5""\t""I6""\t""I7""\t""I8""\t""I9""\t""I10""\t""J1""\t""J2""\t""J3""\t""J4""\t""J5""\t""J6""\t""J7""\t""J8""\t""J9""\t""J10""\t""K1""\t""K2""\t""K3""\t""K4""\t""K5""\t""K6""\t""K7""\t""K8""\t""K9""\t""K10""\t""L1""\t""L2""\t""L3""\t""L4""\t""L5""\t""L6""\t""L7""\t""L8""\t""L9""\t""L10""\t""M1""\t""M2""\t""M3""\t""M4""\t""M5""\t""M6""\t""M7""\t""M8""\t""M9""\t""M10""\t""N1""\t""N2""\t""N3""\t""N4""\t""N5""\t""N6""\t""N7""\t""N8""\t""N9""\t""N10""\t""O1""\t""O2""\t""O3""\t""O4""\t""O5""\t""O6""\t""O7""\t""O8""\t""O9""\t""O10""\t""P1""\t""P2""\t""P3""\t""P4""\t""P5""\t""P6""\t""P7""\t""P8""\t""P9""\t""P10""\t""age""\t""gender""\t""accuracy""\t""country""\t""source""\t""elapsed"""
0,1\t4\t2\t3\t3\t2\t3\t4\t4\t3\t4\t4\t5\t4\t5\t4...
1,4\t3\t4\t3\t4\t4\t4\t4\t2\t2\t4\t4\t4\t4\t5\t4...
2,3\t4\t4\t4\t4\t4\t4\t3\t2\t2\t4\t4\t5\t5\t4\t4...
3,4\t5\t4\t4\t4\t3\t3\t2\t2\t2\t4\t2\t4\t5\t4\t5...
4,4\t0\t4\t4\t4\t3\t5\t1\t2\t4\t2\t4\t4\t5\t5\t4...
...,...
49154,1\t1\t1\t1\t1\t1\t1\t1\t1\t1\t1\t1\t1\t1\t1\t1...
49155,4\t5\t4\t4\t4\t3\t5\t3\t2\t3\t4\t4\t5\t5\t4\t4...
49156,2\t4\t2\t3\t3\t3\t4\t4\t2\t4\t2\t4\t4\t4\t4\t4...
49157,4\t4\t4\t4\t4\t4\t4\t4\t1\t3\t4\t3\t4\t5\t3\t4...
